In [ ]:
1/2*((0.8367*0.9111)/(0.83670+.9111))

In [ ]:
!git clone https://github.com/januar-png/earlystop.git

In [1]:
!pip install jcopdl

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for jcopdl: filename=jcopdl-2.3.1-py2.py3-none-any.whl size=32805 sha256=2640a38cce7e8039fac4fbd3c7f01a64ff302a3e2f6865dd1e5d6a38ad71da78
  Stored in directory: /root/.cache/pip/wheels/e3/09/97/1271913ce50297b4cb5cdaf1b9dc54814d9d60437864bea0ce
  Created wheel for pyperclip: filename=pyperclip-1.9.0-py3-none-any.whl size=11002 sha256=5cfc221cb0f8c3a9a49d8bcfde6cda0f27cebf1ed56311bba36ffa5068aa9ab5
  Stored in directory: /root/.cache/pip/wheels/cc/ae/36/ee17d1de094fcb61e24106cb329b5103861e819f94bef5e10a
Successfully built jcopdl pyperclip


In [2]:
from earlystop import Earlystop

ModuleNotFoundError: No module named 'earlystop'

In [2]:
import torch
from torch import nn, optim
import numpy as np
import matplotlib.pyplot as plt
import os

In [3]:
from torchvision import datasets,transforms
from torch.utils.data import DataLoader

In [4]:
batch_size = 64
img_size = 224, 224

train_transform = transforms.Compose([
    transforms.RandomRotation(7),
    transforms.RandomResizedCrop(img_size, scale=(0.8, 1)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor()
])

val_transform = transforms.Compose([
    transforms.Resize(size=(img_size)),
    transforms.ToTensor()
])

train_set = datasets.ImageFolder('/kaggle/input/penyakit-padi/train', transform = train_transform)
trainloader = DataLoader(train_set, batch_size, shuffle = True)

val_set = datasets.ImageFolder('/kaggle/input/penyakit-padi/val', transform = val_transform)
valloader = DataLoader(val_set, batch_size, shuffle = False)

In [ ]:
feature, labels = next(iter(trainloader))
print(feature.shape)

In [ ]:
n1, n2 = max(0, 1 - 0.3), 1 + 0.3
print(n1, n2)

In [5]:
label2cat = train_set.classes

In [ ]:
fig, axes = plt.subplots(7, 7, figsize=(20,20))
for img,label, ax in zip(feature,labels, axes.flatten()):
    ax.imshow(img.permute(1, 2, 0).cpu())
    ax.set_title(f'{label2cat[label.item()]}')
    ax.axis('off')
plt.savefig('preprocessing.png')

In [ ]:
plt.savefig('preprocessing.png')

In [6]:
from jcopdl.callback import Callback

In [7]:
from tqdm import tqdm

In [8]:
def loop_fn(mode, dataset, dataloader, model, criterion, optimizer, device):
    if mode == "train":
        model.train()
    elif mode == "val":
        model.eval()
    cost = correct = 0
    for feature, target in tqdm(dataloader, desc=mode.title()):
        feature, target = feature.to(device), target.to(device)
        output = model(feature)
        loss = criterion(output, target)
        
        if mode == "train":
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        cost += loss.item() * feature.shape[0]
        correct += (output.argmax(1) == target).sum().item()
    cost = cost / len(dataset)
    acc = correct / len(dataset)
    return cost, acc

In [56]:
def fit(train_set, trainloader, val_set, valloader, model, criterion, optimizer, device, callback, epochs):
    for epoch in range(epochs):
        model.train()
        train_loss, train_score = loop_fn("train", train_set, trainloader, model, criterion, optimizer, device)
        
        model.eval()
        with torch.no_grad():
            val_loss, val_score = loop_fn("val", val_set, valloader, model, criterion, optimizer, device)

        callback.log("train_loss", train_loss)
        callback.log("train_score", train_score)
        callback.log("val_loss", val_loss)
        callback.log("val_score", val_score)

        if callback.early_stopping("minimize", monitor="val_loss") or epoch >= epochs - 1:
            callback.add_plot(["train_loss", "val_loss"], scale="semilogy")
            callback.add_plot(["train_score", "val_score"], scale="linear")
            break

In [49]:
!git clone https://github.com/januar-png/Model.git

fatal: destination path 'Model' already exists and is not an empty directory.


In [50]:
from Model.CNNpenyakitPadi import CNNpenyakitPadi

In [57]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNNpenyakitPadi(output_size = len(train_set.classes)).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=2e-6)
callback = Callback(model, optimizer,early_stop_patience=2)
epoch = 100

In [58]:
fit(train_set, trainloader, val_set, valloader, model, criterion, optimizer, device, callback, epoch)

Val: 100%|██████████| 6/6 [00:17<00:00,  2.94s/it]


,train_loss,train_score,val_loss,val_score
epoch,,,,
66,0.865082,0.896825,0.863659,0.900000
67,0.863499,0.898413,0.861953,0.902778
68,0.860844,0.901587,0.861631,0.905556
69,0.863501,0.896032,0.863803,0.900000
70,0.862248,0.901587,0.861126,0.902778
71,0.856706,0.908730,0.860314,0.902778
72,0.858617,0.898413,0.858748,0.902778
73,0.857189,0.908730,0.857106,0.902778
74,0.851360,0.909524,0.859086,0.900000


Val: 100%|██████████| 6/6 [00:17<00:00,  2.98s/it]


==> Execute Early Stopping at epoch: 75 | Best val_loss: 0.8571
==> Best model is saved at output


In [ ]:
!pip install torchsummary

In [ ]:
45*4

In [ ]:
test_transform = transforms.Compose([
    transforms.Resize(size=(224,224)),
    transforms.ToTensor()
])
test_set = datasets.ImageFolder('/kaggle/input/penyakit-padi/test', transform=test_transform)
testloader = DataLoader(test_set, shuffle=False, batch_size=180)

In [ ]:
feature, target = next(iter(testloader))
feature, target = feature.to(device), target.to(device)
print(feature.shape)

In [ ]:
with torch.no_grad():
    model.load_state_dict(torch.load('/kaggle/working/model/best_model.pth'))
    model.eval()
    output = model(feature)
    preds = output.argmax(1)
preds

In [ ]:
label2cat = train_set.classes

fig, axes = plt.subplots(15,12, figsize=(40,40))
for img, label, pred, ax in zip(feature, target, preds, axes.flatten()):
    ax.imshow(img.permute(1, 2, 0).cpu())
    font = {"color":'r'} if label.item() != pred.item() else {"color": 'g'}
    label, pred = label2cat[label.item()], label2cat[pred.item()]  
    ax.set_title(f"label: {label} | pred: {pred}", fontdict=font)
    ax.axis('off')

In [ ]:
180/15

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNNpenyakitPadi(output_size = len(train_set.classes)).to(device)
criterion = nn.CrossEntropyLoss()

In [ ]:
def evaluate_model(model, testloader, criterion, device):
    model.load_state_dict(torch.load('/kaggle/working/model/best_model.pth'))
    model.eval()
    total_loss = 0.0
    all_labels = []
    all_predictions = []

    with torch.no_grad():
        for inputs, labels in testloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            total_loss += loss.item()

            _, predictions = torch.max(outputs, 1)
            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predictions.cpu().numpy())

    average_loss = total_loss / len(testloader)
    accuracy = sum(np.array(all_labels) == np.array(all_predictions)) / len(all_labels)
    
    class_counts_labels = np.bincount(all_labels)
    class_counts_predictions = np.bincount(all_predictions)

    print(f'Test Loss: {average_loss:.4f}, Test Accuracy: {accuracy:.4f}')

    print("Classification Report:")
    print(classification_report(all_labels, all_predictions))

    print("Confusion Matrix:")
    cm = confusion_matrix(all_labels, all_predictions)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='g', cmap='Blues', xticklabels=test_set.classes, yticklabels=test_set.classes)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix')
    plt.show()
    
    if not os.path.exists('confusion_matrix'):
        os.makedirs('confusion_matrix')

    plt.savefig(f'confusion_matrix/{model.__class__.__name__}.png')

In [ ]:
evaluate_model(model, testloader, criterion, device)

In [ ]:
from torchsummary import summary

model = CNNpenyakitPadi(output_size = 6).to(device)
summary(model, (3, 224, 224))

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from PIL import Image

output_size = 4  
model = CNNpenyakitPadi(output_size)

def visualize_feature_maps(model, layer_idx, input_image):
    activations = model.feature[layer_idx].block(input_image)
    activations = activations.detach()

    fig, axs = plt.subplots(activations.size(1))
    fig.suptitle('Feature Map Visualization (Layer {})'.format(layer_idx))

    for i, ax in enumerate(axs):
        ax.imshow(activations[0, i, :, :].cpu(), cmap='gray')  
        ax.axis('off')
        print(f"feature map: {activations}")

    plt.show()

def visualize_relu(model, layer_idx, input_image):
    activations = model.feature[layer_idx].block(input_image)
    relu_activations = F.relu(activations)
    relu_activations = relu_activations.detach()

    fig, axs = plt.subplots(relu_activations.size(1))
    fig.suptitle('ReLU Visualization (Layer {})'.format(layer_idx))

    for i, ax in enumerate(axs):
        ax.imshow(relu_activations[0, i, :, :].cpu(), cmap='gray')  
        ax.axis('off')
        print(f"ReLU activation: {relu_activations}")

    plt.show()

def visualize_bias(model, layer_idx):
    layer = model.feature[layer_idx].block.conv2d  # Assuming bias is in Conv2d layer
    bias = layer.bias.data

    fig, ax = plt.subplots()
    fig.suptitle('Bias Visualization (Layer {})'.format(layer_idx))

    ax.plot(bias.cpu().numpy())
    ax.set_xlabel('Filter Index')
    ax.set_ylabel('Bias Value')
    print(f"bias: {bias}")
    plt.show()

def visualize_input_channels(input_image):
    channels = input_image.squeeze(0).permute(1, 2, 0).cpu().numpy()

    fig, axs = plt.subplots(channels.shape[2])
    fig.suptitle('Input Image Channels Visualization')

    for i, ax in enumerate(axs):
        ax.imshow(channels[:, :, i], cmap='gray')  
        ax.axis('off')
        print(f"channel {i + 1}: {channels[:, :, i]}")

    plt.show()

image_path = '/kaggle/input/rice-desease/datapadi/test/brown_spot/brown_val (50).jpg' 
input_image = Image.open(image_path)
transform = transforms.Compose([
    transforms.Resize((6, 6)),  
    transforms.ToTensor()
])
input_image = transform(input_image).unsqueeze(0)

visualize_input_channels(input_image)

visualize_feature_maps(model, 0, input_image)

visualize_relu(model, 0, input_image)

visualize_bias(model, 0)

In [ ]:
from PIL import Image

model = CNNpenyakitPadi(output_size = len(train_set.classes)).to(device)
model.load_state_dict(torch.load('/kaggle/working/model/best_model.pth'))
model.eval()

activation1 = {}
jumlah = 100

def get_activation(name):
    def hook(module, input, output):
        activation1[name] = output.detach()
    return hook

for name, layer in model.named_modules():
    if isinstance(layer, (nn.Conv2d, nn.ReLU)):
        layer.register_forward_hook(get_activation(name))

img = Image.open('/kaggle/input/padi-data/test/bercak/IMG000000000013.jpg')
rgb_image = img.convert('RGB')
transform = transforms.Compose([
    transforms.Resize(size=(224, 224)),
    transforms.ToTensor()
])
img_tensor = transform(rgb_image)
img_tensor = img_tensor.unsqueeze(0).to(device)

with torch.no_grad():
    output = model(img_tensor)
    
for name in activation1:
    folder_path = os.path.join('custom_12', name)
    file = name

    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

    if file in activation1:
        num_images = min(max(3, activation1[file].shape[1]), 10)
        for i in range(num_images):
            current_image = activation1[file][0, i].cpu().numpy()
            print(f"Shape of {file}-{i}: {current_image.shape}")
            
            fig, ax = plt.subplots(1, 1)
            ax.imshow(current_image)
            ax.axis('off')
            plt.tight_layout()

            fig.savefig(f'{folder_path}/{file}-{i}.png', dpi=300, transparent=True)

In [ ]:
test_transform = transforms.Compose([
    transforms.Resize(size=(224,224)),
    transforms.ToTensor()
])
test_set = datasets.ImageFolder('/kaggle/input/penyakit-padi/test', transform=test_transform)
testloader = DataLoader(test_set, shuffle=False, batch_size=180)

feature, target = next(iter(testloader))
feature, target = feature.to(device), target.to(device)
print(feature.shape)

In [ ]:
class_names = test_set.classes
for index, class_name in enumerate(class_names):
    print(f"name {class_name} label = {index}")

In [ ]:
for data_index, (sample, label) in enumerate(test_set):
    class_name = test_set.classes[label]
    print(f"Index: {data_index}, Class Name: {class_name}, Label: {label}")

In [ ]:
0.86695 +0.032092+ 0.051306+ 0.049649

In [ ]:

with torch.no_grad():
    model = CNNpenyakitPadi(output_size = len(train_set.classes)).to(device)
    model.load_state_dict(torch.load('/kaggle/working/model/best_model.pth'))
    model.eval()
    output = model(feature)
    preds = output.argmax(1)
print(output)

In [ ]:
print(preds)

In [ ]:
x = np.arange(2,10,2)
y = x**2
pr

In [ ]:
fig, axes = plt.subplots(1,2)
axes[0].plot(x,y)
axes[1].plot(y,x)